THIS JUPYTER NOTEBOOK IS INTENDED TO BE AN EXAMPLE FOR THE USAGE OF THE FUNCTIONS PRESENTS IN THIS REPO.

## Data Download
First of all please, download those files and unzip them in the same directory of this file:

ICBM MNI 152 2009a: http://www.bic.mni.mcgill.ca/~vfonov/icbm/2009/mni_icbm152_nlin_sym_09a_nifti.zip

|X| Datasets: T1:   http://biomedic.doc.ic.ac.uk/brain-development/downloads/IXI/IXI-T1.tar
              


## Read files
The first need is to read files

In [1]:
from tkinter import filedialog
import itk

t1_filename = filedialog.askopenfilename(title = 'Select the T1-W image')
t1 = itk.imread(t1_filename, itk.F)

atlas_filename = filedialog.askopenfilename(title = 'Select the Atlas image')
atlas = itk.imread(atlas_filename, itk.F)

## Register the Atlas image
If we want to register an atlas over an image with a multimap (Rigid -> Affine -> BSpline) registration:

In [2]:
from Neuroradiomics.registration import elastix_multimap_registration
registration_object = elastix_multimap_registration (t1, atlas)
registered_atlas = registration_object.GetOutput()

itk.imwrite(registered_atlas, 'registered_atlas.nii') #to write on the disk the image

Here is reported an example of the MNI 152 t1 atlas registered on the 002-Guys-0828-T1 scan of the IXI database.

This is the original scan:

<img src="./Images/T1.jpg" alt= “T1W_scan” width="50%" height="50%" title="T1W original scan">

This is the atlas registered on the scan:

<img src="./Images/registered_atlas.jpg" alt= “” width="50%" height="50%">

## Skull Stripping an Image

In this case there is the need to read the brain mask of the atlas

In [4]:
from Neuroradiomics.skull_stripping import skull_stripper

atlas_brain_mask_filename = filedialog.askopenfilename(title = 'Select the brain mask from the atlas')
atlas_brain_mask = itk.imread(atlas_brain_mask_filename, itk.F)

brain = skull_stripper(t1, atlas, atlas_brain_mask)

itk.imwrite(brain, 'brain.nii') 

Here it is possible to see the brain extracted from the IXI002-Guys-0828-T1 scan from the IXI database.

<img src="./Images/brain.jpg" alt= “” width="50%" height="50%">

There is also the possibility to obtain the brain mask and eventually the transformation applied to the object

In [5]:
from Neuroradiomics.skull_stripping import skull_stripping_mask

brain_mask, transform = skull_stripping_mask(t1, atlas, atlas_brain_mask, transformation_return = True)

There is no need to apply those two functions if are required both brain and the brain mask.
There is a masking function that can apply directly the mask.

In [ ]:
from Neuroradiomics.skull_stripping import negative_3d_masking

brain = negative_3d_masking (t1, brain_mask)

## Segmenting the tissues in a brain image

In this case is necessary to read the probability masks of the atlas and register them to the T1 space.
Since the atlas is already registered we only transform them using the already obtained transform.
In order to do this we will use the transformix filter.

In [6]:
from Neuroradiomics.segmentation import brain_segmentation

atlas_wm_filename = filedialog.askopenfilename(title = 'Select the White Matter probability map')
atlas_wm = itk.imread(atlas_wm_filename, itk.F)

atlas_gm_filename = filedialog.askopenfilename(title = 'Select the Grey Matter probability map')
atlas_gm = itk.imread(atlas_gm_filename, itk.F)

atlas_csf_filename = filedialog.askopenfilename(title = 'Select the Cerebrospinal Fluid probability map')
atlas_csf = itk.imread(atlas_csf_filename, itk.F)

reg_wm = itk.transformix_filter(atlas_wm, transform)
reg_gm = itk.transformix_filter(atlas_gm, transform)
reg_csf = itk.transformix_filter(atlas_csf, transform)
                                                

labels = brain_segmentation ( brain, brain_mask, reg_wm, reg_gm, reg_csf, auto_mean = True)

itk.imwrite(labels, 'labels.nii')

Labels are so distinguished:

0. Background
1. Cerebrospinal Fluid
2. Grey Matter
3. White Matter

Here is provided and example of segmentation for the IXI002-Guys-0828-T1 scan from the IXI database.
The color map is the subsequent:
* Red:    White Matter
* Yellow: Grey Matter
* Green:  Cerebrospinal Fluid

<img src="./Images/segmentation.jpg" alt= “” width="50%" height="50%">

There is also a function that permits to choose and keep only one label.

In [8]:
from Neuroradiomics.segmentation import label_selection


white_matter_label = label_selection(labels, 3) #in order to select only the white matter

itk.imwrite(white_matter_label, 'white_matter_label.nii') #wirte down the selected label image

This is the final result. Here is reported the automatically segmented white matter on the original T1W scan.

<img src="./Images/WM.jpg" alt= “” width="50%" height="50%">